In [1]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"  "einops" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Dataset Processing

In [2]:
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [3]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/instruct_data_chatgpt_generate.csv')
df.head()

,Instruction,Context,Response,Category
0,Use the Input and generate the suitable catego...,"Lions, Tigers, Cheetahs, Leopards",Big Cats,category_generation
1,Use the Input and generate the suitable catego...,"Piano, Guitar, Violin, Trumpet",Musical Instruments,category_generation
2,Use the Input and generate the suitable catego...,"Apple, Banana, Cherry, Orange",Fruits,category_generation
3,Use the Input and generate the suitable catego...,"Toyota, Ford, BMW, Tesla",Car Brands,category_generation
4,Use the Input and generate the suitable catego...,"Python, Java, C++, Ruby",Programming Languages,category_generation


In [4]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Instruction', 'Context', 'Response', 'Category'],
    num_rows: 60
})

In [5]:
def apply_prompt_template(examples):
  instruction = examples['Instruction']
  context = examples.get("Context")
  response = examples["Response"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
  else:
    full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [6]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and generate the suitable category name

Input:
Lions, Tigers, Cheetahs, Leopards

### Response:
Big Cats

### End


# Finetune model

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [10]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [11]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [12]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [13]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.335200
2,3.167000
3,2.889900
4,2.509300
5,2.125400
6,1.683300
7,1.431800
8,1.073000
9,0.940900
10,0.786700


TrainOutput(global_step=30, training_loss=0.9217781315247218, metrics={'train_runtime': 143.5192, 'train_samples_per_second': 3.345, 'train_steps_per_second': 0.209, 'total_flos': 697828841020416.0, 'train_loss': 0.9217781315247218, 'epoch': 8.0})

In [14]:
from transformers import pipeline

# Keywords Extraction

In [15]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and generate the suitable keywords

Input:
we are doing a research in deep learning for instruct fine tune

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and generate the suitable keywords

Input:
we are doing a research in deep learning for instruct fine tune

### Response:
deep learning, research, instruction, suitable keywords


None


# Category generate

In [16]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and generate the suitable category name

Input:
Jaffna, Kandy, Colombo

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and generate the suitable category name

Input:
Jaffna, Kandy, Colombo

### Response:
Sri Lankan Cities


None


# Group by similarity

In [17]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and group by similarity

Input:
google, CNN, LSTM, yahoo, IOS, Jaffna, Colombo, Android

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Use the Input and group by similarity

Input:
google, CNN, LSTM, yahoo, IOS, Jaffna, Colombo, Android

### Response:
[google, CNN, yahoo, LSTM, Jaffna, Colombo], [iOS, Android]


None
